## IMPORT

In [1]:
import twitter
import spacy
import nltk
from spacy import displacy
import markovify
import gzip
import re
import json
import spacy

In [26]:
nlp = spacy.load('en_core_web_lg')

## RETRIEVE TWEETS: API

In [3]:
twitter_credentials = {
    'consumer_key':        'wbz78wFd0ywcShiTvqgDUV2ry',
    'consumer_secret':     '2qj0P3fygqa0n2LqU6M8LV485OWIAvXWEQOEVLWFNUBdKDcgjz',
    'access_token_key':    '80578720-t7bH4zwD6Q6sUQEFeCb8211wH04Y9ul0EWECo2ofU',
    'access_token_secret': 'PSsX8R4agpxAII9XCYHqE74KObPRWfl9tdG4Xd07olOn6'
}

In [4]:
twapi = twitter.Api(consumer_key=twitter_credentials['consumer_key'],
                    consumer_secret=twitter_credentials['consumer_secret'],
                    access_token_key=twitter_credentials['access_token_key'],
                    access_token_secret=twitter_credentials['access_token_secret'],
                    tweet_mode='extended')

In [5]:
# screen_name = 'nigel_farage'
screen_name = 'realdonaldtrump'
# screen_name = 'borisjohnson'

In [6]:
#  Retrieve as much tweet history as possible for training purposes
timeline = []
max_id = None
while True:
    print('max_id for iteration: {}'.format(max_id))
    tweets = twapi.GetUserTimeline(screen_name=screen_name, include_rts=False, count=200, max_id=max_id)
    timeline += tweets
    
    next_max_id = min(tweets, key=lambda t: t.id).id
    if next_max_id == max_id:
        break
    max_id = next_max_id

max_id for iteration: None
max_id for iteration: 1321858569466114048
max_id for iteration: 1320585745032269825
max_id for iteration: 1319044139208540160
max_id for iteration: 1317056094548774914
max_id for iteration: 1315836801354027008
max_id for iteration: 1314347971048284165
max_id for iteration: 1312864232711520257
max_id for iteration: 1310769062264307715
max_id for iteration: 1308831937734340609
max_id for iteration: 1306693977723342848
max_id for iteration: 1305171136087388160
max_id for iteration: 1303665208322412544
max_id for iteration: 1302586046551597061
max_id for iteration: 1300840191771340801
max_id for iteration: 1299559244593860608
max_id for iteration: 1297706044236472320
max_id for iteration: 1296590565136621568


## RETRIEVE TWEETS: FILE

In [48]:
tweetfile = '/Users/liangjh/Workspace/corpii/tweets-realdonaldtrump.json'
screen_name = 'realdonaldtrump'

In [49]:
with open(tweetfile) as json_file:
    timeline = json.load(json_file)

In [50]:
timeline = [tle for tle in timeline if tle['created_at'][-4:] in ['2020', '2019', '2018', '2017']]

In [51]:
len(timeline)

15698

## PROCESS TWEETS

In [17]:
# timeline.reverse()
# Timeline retrieved from twitter api
timeline_abridged = [{k:v for k,v in elem.AsDict().items() if k in ['created_at', 'full_text', 'id']} 
                      for elem in timeline]

# Timeline retrieved from archive
# timeline_abridged = [{k:v for k,v in elem.items() if k in ['created_at', 'text', 'id']} 
#                       for elem in timeline]

In [19]:
#  Save abridged 
with open('/Users/liangjh/workspace/corpii/tweets-{}.json'.format(screen_name), 'w') as f:
     json.dump({'data': timeline_abridged}, f)

In [21]:
timeline_text_sanitized = [re.sub(r'(http|https)?:\/\/.*[\r\n]*', '', tline['full_text']) # text or full_text
                           for tline in timeline_abridged]

In [22]:
timeline_merged_corpus = ' '.join(timeline_text_sanitized)

In [9]:
# with open('/Users/liangjh/workspace/corpii/tweets-trump.txt') as f:
#     tweets_raw_curr = f.read()

In [25]:
#  Parsed document
doc = nlp(timeline_merged_corpus)

NameError: name 'nlp' is not defined

In [73]:
# tweets_combined = ' '.join(tweets_raw)
# doc = nlp(tweets_combined)
# len(tweets_raw)
# len([token for token in doc])

In [23]:
class POSifiedText(markovify.Text):
    def word_split(self, sentence):
        return ['::'.join((word.orth_, word.pos_)) for word in nlp(sentence)]
    def word_join(self, words):
        sentence = ' '.join(word.split('::')[0] for word in words)
        return sentence

In [29]:
tweet_model = POSifiedText(timeline_merged_corpus, state_size=2)

In [60]:
# for i in range(20):
#     print(tweet_model.make_short_sentence(max_chars=150))

In [30]:
with gzip.GzipFile('/Users/liangjh/workspace/corpii/trumpspire-model-02.gz'.format(screen_name), 'w') as fo:
    fo.write(json.dumps(tweet_model.to_json()).encode('utf-8'))